In [20]:
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo

In [21]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define database and collection
db = client.troubles_db
collection = db.items

In [22]:
#spacy function to return entity info
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')

In [23]:
year = '1969'
combinedurl = 'https://cain.ulster.ac.uk/sutton/chron/'+year+'.html'
tables = pd.read_html(combinedurl)
df = tables[1]
scrapedf = df.iloc[:,1]
len(scrapedf)

16

In [24]:
#list of years
#for loop to run scrape for each year
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=True)
#year = '1969'
#url = 'https://cain.ulster.ac.uk/sutton/chron/'+year+'.html'
#browser.visit(durl)
#
#time.sleep(1)
#
#html = browser.html 
#soup = bs(html,"html.parser")
#
#find titles
#titles = soup.find_all('b')
#
#for title in titles:
#    print(title.prettify())


In [25]:
def text_cleanup(dfrow):
    death = str(scrapedf[dfrow])
    deathsplitspace = death.split()


    #dateofdeath string established and parsed
    deathindex = deathsplitspace.index(year)
    dateofdeath = []
    for deathsplitspaceindex in range(deathindex+1):
        dateofdeath.append(deathsplitspace[deathsplitspaceindex])
    #???:how to convert 'dateofdeath' from string into datetime
    print(dateofdeath)

    #namestring established
    indices = [i for i, s in enumerate(deathsplitspace) if '(' in s]
    namestring = []
    lastname = []
    firstname = []
    namerange = range(deathsplitspaceindex+1,indices[0])
    for namecomps in range(deathsplitspaceindex+1,indices[0]):
        namestring.append(deathsplitspace[namecomps])

    #namestring parsing    
    for namecomp in namestring:
        if namecomp.endswith(",") == True:
            lastnameendvalue = namecomp
            lastnameendindex = namestring.index(lastnameendvalue)
    for lastnameind in range(lastnameendindex+1):
        lastname.append(namestring[lastnameind])

    if lastnameendindex+1 == len(namestring)-1:
        firstname.append(namestring[lastnameendindex+1])
    else:
        for firstnameind in range(lastnameendindex+1,len(namestring)-1):
            firstname.append(namestring[firstnameind])
    lastname = [s.replace(",", "") for s in lastname]
    print(lastname)
    print(firstname)

    #affiliation established
        #excluding for now

    #'killed by' established
    killedbystartindices = [i for i, s in enumerate(deathsplitspace) if 'Killed' in s]
    killedbyendindices = [i for i, s in enumerate(deathsplitspace) if ')' in s]
    killedbystartindice = killedbystartindices[0]
    killedbyendindice = killedbyendindices[2]
    killedby = []

    for killedbycomps in range(killedbystartindice+2,killedbyendindice):
        killedby.append(deathsplitspace[killedbycomps])

    print(killedby)

    #details established
    details = []
    for detailcomp in range(killedbyendindice, len(deathsplitspace)):
        details.append(deathsplitspace[detailcomp])

    print(details)
    
    post = {
                'Date of Death': dateofdeath,
                'First Name': firstname,
                'Last Name': lastname,
                'Killed by:': killedby,
                'Details':details
            }

    collection.insert_one(post)



In [26]:
#passing details through NER
parsed_details = NER(' '.join(details))

for NamedEntity in parsed_details.ents:
    show_ents(NamedEntity)

NameError: name 'details' is not defined

In [27]:
yearsofdeaths = []
for i in range(1969,2002):
    yearsofdeaths.append(str(i))
    
    
for year in yearsofdeaths:
    combinedurl = 'https://cain.ulster.ac.uk/sutton/chron/'+str(year)+'.html'
    tables = pd.read_html(combinedurl)
    df = tables[1]
    scrapedf = df.iloc[:,1]
    for row in range (len(scrapedf)):
        text_cleanup(row)

['14', 'July', '1969']
['McCloskey']
['Francis']
['Royal', 'Ulster', 'Constabulary']
['(RUC)Died', 'one', 'day', 'after', 'being', 'hit', 'on', 'head', 'with', 'batons', 'during', 'street', 'disturbances,', 'Dungiven,', 'County', 'Derry.']
['17', 'July', '1969']
['Devenny']
['Samuel']
['Royal', 'Ulster', 'Constabulary']
['(RUC)Died', 'three', 'months', 'after', 'being', 'badly', 'beaten', 'in', 'his', 'home,', 'William', 'Street,', 'Bogside,', 'Derry.', 'He', 'was', 'injured', 'on', '19', 'April', '1969.']
['14', 'August', '1969']
['Gallagher']
['John']
['Ulster', 'Special', 'Constabulary']
['(USC)Shot', 'during', 'street', 'disturbances,', 'Cathedral', 'Road,', 'Armagh.']
['14', 'August', '1969']
['Rooney']
['Patrick']
['Royal', 'Ulster', 'Constabulary']
['(RUC)Shot', 'at', 'his', 'home,', 'during', 'nearby', 'street', 'disturbances,', 'St', "Brendan's", 'Path,', 'Divis', 'Flats,', 'Belfast.']
['15', 'August', '1969']
['Roy']
['Herbert']
['non-specific', 'Republican', 'group']
['(REP)

['27', 'January', '1971']
['Kavanagh']
['John']
['Irish', 'Republican', 'Army']
['(IRA)Found', 'shot', 'by', 'Blackstaff', 'River,', 'off', 'Roden', 'Street,', 'Belfast.']
['06', 'February', '1971']
['Watt']
['Bernard']
['British', 'Army']
['(BA)Shot', 'during', 'street', 'disturbances,', 'Chatham', 'Street,', 'Ardoyne,', 'Belfast.']
['06', 'February', '1971']
['Saunders']
['James']
['British', 'Army']
['(BA)Shot', 'during', 'gun', 'battle,', 'Louisa', 'Street,', 'off', 'Oldpark', 'Road,', 'Belfast.']
['06', 'February', '1971']
['Curtis']
['Robert']
['Irish', 'Republican', 'Army']
['(IRA)Shot', 'by', 'sniper', 'while', 'on', 'British', 'Army', '(BA)', 'foot', 'patrol,', 'New', 'Lodge', 'Road,', 'Belfast.']
['07', 'February', '1971']
['Bell']
['Albert']
['not', 'known']
['(nk)Found', 'shot', 'by', 'the', 'side', 'of', 'the', 'Belfast', 'to', 'Crumlin', 'Road,', 'Ballyhill,', 'near', 'Belfast,', 'County', 'Antrim.']
['09', 'February', '1971']
['Eakins']
['John']
['Irish', 'Republican', '

['05', 'January', '1972']
['Bryan']
['Keith']
['Irish', 'Republican', 'Army']
['(IRA)Shot', 'by', 'sniper', 'while', 'on', 'British', 'Army', '(BA)', 'foot', 'patrol,', 'Ardmoulin', 'Street,', 'Lower', 'Falls,', 'Belfast.']
['07', 'January', '1972']
["O'Neill"]
['Daniel']
['British', 'Army']
['(BA)Died', 'two', 'days', 'after', 'being', 'shot', 'during', 'gun', 'battle,', 'Oranmore', 'Street,', 'Falls,', 'Belfast.']
['08', 'January', '1972']
['Woods']
['Peter']
['non-specific', 'Loyalist', 'group']
['(LOY)Shot', 'at', 'his', 'home,', 'Lowwood', 'Park,', 'Skegoneill,', 'Belfast.']
['12', 'January', '1972']
['Denham']
['Raymond']
['Irish', 'Republican', 'Army']
['(IRA)Off', 'duty', 'reservist.', 'Shot', 'at', 'his', 'workplace,', 'Waterford', 'Street,', 'Lower', 'Falls,', 'Belfast.']
['13', 'January', '1972']
['Crawford']
['Maynard']
['Irish', 'Republican', 'Army']
['(IRA)Off', 'duty.', 'Shot', 'while', 'driving', 'his', "firm's", 'van', 'along', "King's", 'Road,', 'off', 'Doagh', 'Road,

['01', 'January', '1973']
['Boyce']
['Oliver']
['Ulster', 'Defence', 'Association']
['(UDA)Found', 'shot', 'in', 'ditch,', 'Birdstown,', 'near', 'Burnfoot,', 'County', 'Donegal.']
['01', 'January', '1973']
['Porter']
['Breige']
['Ulster', 'Defence', 'Association']
['(UDA)Found', 'shot', 'in', 'ditch,', 'Birdstown,', 'near', 'Burnfoot,', 'County', 'Donegal.']
['02', 'January', '1973']
['Mooney']
['John']
['non-specific', 'Loyalist', 'group']
['(LOY)Shot', 'outside', 'his', 'workplace,', 'Rolls', 'Royce', 'factory,', 'Upper', 'Newtownards', 'Road,', 'Dundonald,', 'Belfast.']
['04', 'January', '1973']
['Hood']
['James']
['Irish', 'Republican', 'Army']
['(IRA)Off', 'duty.', 'Shot', 'outside', 'his', 'home,', 'Straidarran,', 'near', 'Feeny,', 'County', 'Derry.']
['05', 'January', '1973']
['Rankin']
['Trevor']
['Irish', 'Republican', 'Army']
['(IRA)Shot', 'at', 'Ben', 'Madigan', 'filling', 'station,', 'Shore', 'Road,', 'Belfast.', 'Mistaken', 'for', 'off', 'duty', 'Ulster', 'Defence', 'Regim

['01', 'January', '1974']
['Whyte']
['John']
['Irish', 'Republican', 'Army']
['(IRA)Shot', 'during', 'sniper', 'attack', 'on', 'British', 'Army', '(BA)', 'mobile', 'patrol,', 'McClure', 'Street,', 'off', 'Ormeau', 'Road,', 'Belfast.']
['05', 'January', '1974']
['McCullagh']
['Leo']
['non-specific', 'Loyalist', 'group']
['(LOY)Shot', 'at', 'his', 'home,', 'Ravenscroft', 'Avenue,', 'Strandtown,', 'Belfast.']
['10', 'January', '1974']
['Crawford']
['John']
['Ulster', 'Volunteer', 'Force']
['(UVF)Found', 'shot', 'near', 'his', 'workplace,', 'Milltown', 'Row,', 'Falls,', 'Belfast.']
['11', 'January', '1974']
['Byrne']
['Cecilia']
['Official', 'Irish', 'Republican', 'Army']
['(OIRA)Civilian', 'employed', 'by', 'British', 'Army', '(BA).', 'Killed', 'by', 'bomb', 'attached', 'to', 'car', 'which', 'exploded', 'shortly', 'after', 'leaving', 'Ebrington', 'British', 'Army', '(BA)', 'base,', 'Waterside,', 'Derry.']
['11', 'January', '1974']
['Dunne']
['John']
['Official', 'Irish', 'Republican', 'Ar

...

['03', 'January', '1993']
['Shields']
['Patrick']
['Ulster', 'Volunteer', 'Force']
['(UVF)Shot', 'at', 'his', 'home', '/', 'shop,', 'Lisnagleer,', 'near', 'Dungannon,', 'County', 'Tyrone.']
['03', 'January', '1993']
['Shields']
['Diarmuid']
['Ulster', 'Volunteer', 'Force']
['(UVF)Shot', 'at', 'his', 'home', '/', 'shop,', 'Lisnagleer,', 'near', 'Dungannon,', 'County', 'Tyrone.']
['11', 'January', '1993']
['Boyd']
['Matthew']
['Irish', 'Republican', 'Army']
['(IRA)Shot', 'while', 'driving', 'his', 'car', 'along', 'Donaghmore', 'Road,', 'Dungannon,', 'County', 'Tyrone.']
['14', 'January', '1993']
['Butler']
['Anthony']
['Ulster', 'Freedom', 'Fighters']
['(UFF)Former', 'republican', 'prisoner.', 'Shot', 'while', 'in', "friend's", 'home,', 'Agra', 'Street,', 'Ballynafeigh,', 'Belfast.']
['17', 'January', '1993']
['McKenna']
['Sharon']
['Ulster', 'Volunteer', 'Force']
['(UVF)Shot', 'while', 'visiting', "friend's", 'home,', 'Shore', 'Road,', 'Mount', 'Vernon,', 'Belfast.']
['21', 'January', '

['27', 'January', '1994']
['McDermott']
['Cormac']
['Ulster', 'Volunteer', 'Force']
['(UVF)Shot', 'at', 'his', 'home,', 'Fisherwick', 'Gardens,', 'Ballymena,', 'County', 'Antrim.']
['27', 'January', '1994']
['Doherty']
['John']
['Ulster', 'Freedom', 'Fighters']
['(UFF)Shot', 'at', 'his', 'lodgings,', 'Candahar', 'Street,', 'Ballynafeigh,', 'Belfast.']
['27', 'January', '1994']
['Maxwell']
['Robin']
['Royal', 'Ulster', 'Constabulary']
['(RUC)Shot,', 'during', 'attempted', 'robbery', 'at', 'petrol', 'filling', 'station,', 'New', 'Road,', 'Donaghadee,', 'County', 'Down.']
['03', 'February', '1994']
['Sweeney']
['Mark']
['Ulster', 'Volunteer', 'Force']
['(UVF)Taxi', 'driver.', 'Found', 'shot', 'in', 'his', 'car,', 'Ballyreagh', 'Road,', 'Newtownards,', 'County', 'Down.']
['10', 'February', '1994']
['McGlinchey']
['Dominic']
['not', 'known']
['(nk)Originally', 'from', 'Northern', 'Ireland,', '.', 'Former', 'leader', 'of', 'Irish', 'National', 'Liberation', 'Army', '(INLA).', 'Shot', 'while'

['02', 'March', '1995']
['Seymour']
['James']
['Irish', 'Republican', 'Army']
['(IRA)Died', 'nearly', '22', 'years', 'after', 'being', 'shot', 'by', 'sniper,', 'outside', 'Coalisland', 'Royal', 'Ulster', 'Constabulary', '(RUC)', 'base,', 'County', 'Tyrone.', 'Been', 'in', 'a', 'coma', 'since', 'the', 'incident', 'on', '4', 'May', '1973.']
['28', 'April', '1995']
['Mooney']
['Michael']
['Irish', 'Republican', 'Army']
['(IRA)Shot,', 'while', 'in', '18', 'Steps', 'Bar,', 'Ann', 'Street,', 'Belfast.']
['05', 'September', '1995']
['Kane']
['Tony']
['Irish', 'Republican', 'Army']
['(IRA)Shot,', 'while', 'sat', 'in', 'his', 'stationary', 'car,', 'St.', 'Agnes', 'Drive,', 'Andersonstown,', 'Belfast']
['28', 'September', '1995']
['Elliott']
['William']
['Red', 'Hand', 'Commando']
['(RHC)Shot,', 'while', 'leaving', 'friends', 'house,', 'Primacy', 'Park,', 'Bangor,', 'County', 'Down.', 'Internal', 'Red', 'Hand', 'Commando', '(RHC)', 'dispute.']
['27', 'November', '1995']
['Harley']
['Norman']
['n

['10', 'January', '1998']
['Enright']
['Terry']
['Loyalist', 'Volunteer', 'Force']
['(LVF)Security', 'man.', 'Shot', 'outside', 'Space', 'Nightclub,', 'Talbot', 'Street,', 'Belfast.']
['18', 'January', '1998']
['McCusker']
['Fergal']
['Loyalist', 'Volunteer', 'Force']
['(LVF)Found', 'shot,', 'behind', 'youth', 'centre,', 'off', 'Tircane', 'Road,', 'Maghera,', 'County', 'Derry.']
['19', 'January', '1998']
['Guiney']
['Jim']
['Irish', 'National', 'Liberation', 'Army']
['(INLA)Shot', 'at', 'his', 'carpet', 'shop,', 'Kingsway,', 'Dunmurry,', 'near', 'Belfast,', 'County', 'Antrim.']
['19', 'January', '1998']
['Brennan']
['Larry']
['Ulster', 'Defence', 'Association']
['(UDA)Taxi', 'driver.', 'Shot', 'while', 'sitting', 'in', 'his', 'stationary', 'car,', 'outside', 'taxi', 'depot,', 'Ormeau', 'Road,', 'Ballynafeigh,', 'Belfast.']
['21', 'January', '1998']
['Hughes']
['Benedict']
['Ulster', 'Defence', 'Association']
['(UDA)Shot', 'outside', 'his', 'workplace,', 'Utility', 'Street,', 'off', 'Do

['27', 'January', '1999']
['Collins']
['Eamon']
['non-specific', 'Republican', 'group']
['(REP)Found', 'beaten', 'and', 'stabbed', 'to', 'death,', 'at', 'the', 'junction', 'of', 'Watsons', 'Road', 'and', 'Dorans', 'Hill,', 'Newry,', 'County', 'Down.']
['15', 'March', '1999']
['Nelson']
['Rosemary']
['Red', 'Hand', 'Defenders']
['(RHD)Lawyer.', 'Killed', 'by', 'booby', 'trap', 'bomb', 'attached', 'to', 'her', 'car,', 'which', 'exploded', 'shortly', 'after', 'leaving', 'her', 'home,', 'while', 'travelling', 'along', 'Ashford', 'Grange,', 'Lurgan,', 'County', 'Armagh.']
['17', 'March', '1999']
['Curry']
['Frankie']
['Ulster', 'Volunteer', 'Force']
['(UVF)Shot', 'while', 'walking', 'across', 'waste', 'ground,', 'off', 'Malvern', 'Way,', 'Shankill,', 'Belfast.', 'Red', 'Hand', 'Commando', '(RHC)', '/', 'Ulster', 'Volunteer', 'Force', '(UVF)', 'dispute.']
['09', 'May', '1999']
['Fegan']
['Brendan']
['non-specific', 'Republican', 'group']
['(REP)Shot', 'while', 'in', 'Hermitage', 'Bar,', 'Can